# 크롤링

### 크롬 드라이버 다운 필요
(크롬이 최신 버전이라는 전제하에 시작, 작업 디렉토리까지의 절대경로에 한글이 있으면 오류가 발생할 수 있음)
1. 크롬 접속 후 우측 상단 점 3개 클릭 → 도움말 → Chrome 정보 클릭
2. 자신의 Chrome 버전 확인
3. 링크 접속 https://googlechromelabs.github.io/chrome-for-testing/
4. 자신과 똑같거나 거의 비슷한 버전의 왼쪽 단어 클릭(Stable, Beta, Dev, Canary 중 하나)
5. chromedriver 항목중에서 자신의 OS 환경에 맞는 URL 주소 복사
6. 인터넷 창에 URL 복사 붙여넣기하면 다운받을 수 있음
7. 자신의 작업 디렉토리에 압축 해제

In [2]:
# 터미널에 붙여넣기
# pip install bs4 selenium

In [15]:
# 원하는 검색어 설정
search_keyword = "LLM"

# 작업 디렉토리까지의 절대경로 입력 + 마지막에 / 입력
load_folder = 'C:/Users/kimsu/Desktop/crawling/'

# 데이터를 저장할 파일 이름 설정
crawling_data_file_name = f'riss_crawling_{search_keyword}.csv'

# 직접 검색하여 총 몇 페이지인지 확인 후 입력
page_count = 46                                  

In [16]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
import datetime
import os
import getpass
import time

# URL 설정
URL = f"https://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query={search_keyword}&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=re_a_kor&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=search&isFDetailSearch=N&sflag=1&searchQuery=LLM&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&resultKeyword={search_keyword}&pageNumber=1&p_year1=&p_year2=&dorg_storage=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code=&ccl_code=&language=&inside_outside=&fric_yn=&db_type=&image_yn=&regnm=&gubun=&kdc=&ttsUseYn="

In [17]:
# 여러 페이지를 돌면서 링크를 가져오는 함수
def get_paper_links(driver, url):
    driver.get(url)
    time.sleep(3)  # 페이지 로딩을 기다립니다.

    soup = bs(driver.page_source, 'html.parser')
    links = []

    for a in soup.select("#divContent > div > div.rightContent.wd756 > div > div.srchResultW > div.srchResultListW > ul > li > div.cont.ml60 > p.title > a"):
        links.append('https://www.riss.kr' + a['href'])

    return links

# 논문 세부 정보를 가져오는 함수
def get_paper_details(driver, url):
    driver.get(url)
    time.sleep(3)  # 페이지 로딩을 기다립니다.

    soup = bs(driver.page_source, 'html.parser')

    title = soup.select_one('h3.title')
    title_kor = title.get_text(strip=True) if title else ""
    
    authors = soup.select_one('#thesisInfoDiv > div.infoDetail.on > div.infoDetailL > ul > li:nth-of-type(1) > div > p')
    authors_text = authors.get_text(strip=True) if authors else ""
    
    keywords = soup.select_one('#thesisInfoDiv > div.infoDetail.on > div.infoDetailL > ul > li:nth-of-type(7) > div > p')
    keywords_text = keywords.get_text(strip=True) if keywords else ""

    multilingual_abstract = soup.select_one('#additionalInfoDiv > div > div:nth-child(1) > div:nth-child(4) > p')
    multilingual_abstract_text = multilingual_abstract.get_text(strip=True) if multilingual_abstract else ""

    abstract = soup.select_one('#additionalInfoDiv > div > div:nth-child(2) > div:nth-child(4) > p')
    abstract_text = abstract.get_text(strip=True) if abstract else ""

    year = soup.select_one('#thesisInfoDiv > div.infoDetail.on > div.infoDetailL > ul > li:nth-child(5) > div > p')
    year_text = year.get_text(strip=True) if year else ""

    return {
        '저자': authors_text,
        '제목': title_kor,
        '키워드': keywords_text,
        '발행연도': year_text,
        '요약': multilingual_abstract_text + '\n' + abstract_text, # 국문과 영문의 위치가 논문마다 달라서 합침
        '링크': url
    }

# 데이터를 CSV 파일에 저장하는 함수
def save_to_csv(data, csv_path):
    df = pd.DataFrame(data)
    if os.path.isfile(csv_path):
        df.to_csv(csv_path, mode='a', header=False, index=False)
    else:
        df.to_csv(csv_path, mode='w', header=True, index=False)

# 폴더를 생성하는 함수
def make_folder(folder_name):
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

In [18]:
if __name__ == "__main__":
    driver_path = 'chromedriver-win64/chromedriver.exe' # 압축 해제한 폴더
    chrome_service = Service(driver_path)
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("headless")
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

    now = datetime.datetime.now().strftime('%Y-%m-%d')
    user_name = getpass.getuser()
    folder_root = load_folder
    path = folder_root + now + '/'
    make_folder(path)

    csv_path = path + crawling_data_file_name
    all_paper_details = []

    start_page = 1
    end_page = page_count

    for page in range(start_page, end_page + 1):
        iStartCount = (page - 1) * 10  # 페이지당 10개씩
        # pageNumber, iStartCount 값에 맞춰 URL 생성
        URL = (f"https://www.riss.kr/search/Search.do?"
               f"isDetailSearch=N&searchGubun=true&viewYn=OP&query={search_keyword}"
               f"&queryText=&iStartCount={iStartCount}&iGroupView=5&icate=all"
               f"&colName=re_a_kor&exQuery=&exQueryText=&order=%2FDESC&onHanja=false"
               f"&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=search"
               f"&isFDetailSearch=N&sflag=1&searchQuery={search_keyword}"
               f"&fsearchSort=&fsearchOrder=&limiterList=&limiterListText="
               f"&facetList=&facetListText=&fsearchDB=&resultKeyword={search_keyword}"
               f"&pageNumber={page}&p_year1=&p_year2=&dorg_storage=&mat_type="
               f"&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code="
               f"&ccl_code=&language=&inside_outside=&fric_yn=&db_type=&image_yn="
               f"&regnm=&gubun=&kdc=&ttsUseYn=")

        paper_links = get_paper_links(driver, URL)

        # 2) 각 페이지별로 추출한 링크에 대해서 상세 정보 수집
        for link in paper_links:
            paper_details = get_paper_details(driver, link)
            all_paper_details.append(paper_details)

    # 3) 모든 페이지에 대해 수집한 결과를 CSV로 저장
    save_to_csv(all_paper_details, csv_path)

    driver.quit()

# 실행하는 동안 컴퓨터가 절전모드에 들어가면 안됨